In [1]:
from __future__ import print_function, division

In [2]:
import random
import uuid
import os
import sys

### Helper Functions

Import functions for creating nicely formatted output.

In [3]:
# %load 'utils.py'
from IPython.display import display, HTML

def pp_bold(str):
    display(HTML('<b>{}</b>'.format(str)))


def pp_listOflist(l):
    display(HTML(
        u'<table>{}</table>'.format(
            u''.join(u'<tr>{}</tr>'.format(
                u''.join(u'<td>{}</td>'.format(v) for v in sublist)) for sublist in l))))
    

def pp_dict(d, rows=None):
    if not rows or rows >= len(d):
        display(HTML(
            u'<table>{}</table>'.format(
                u''.join(u'<tr><td><b>{}</b></td><td>{}</td></tr>'.format(k, d[k]) for k in d))))
    else:
        nitems = len(d)
        width = -(-nitems // rows)
        i = 0
        list_ = [[] for _ in range(rows)]
        for _ in range(width):
            for row in range(rows):
                if i < nitems:
                    k, v = d.items()[i]
                    list_[row].extend(['<b>{}</b>'.format(k), v])
                i += 1
        pp_listOflist(list_)


def pp_dictOflist(d):
    display(HTML(
        u'<table>{}</table>'.format(
            u''.join(u'<tr><td><b>{}</b></td>{}</tr>'.format(k,
                u''.join(u'<td>{}</td>'.format(v) for v in d[k])) for k in d.keys()))))
    

def pp_dfinfo(df, width=4):
    ncols = len(df.columns)
    width = min(width, ncols)
    depth = -(-ncols // width)
    i = 0
    list_ = [[] for _ in range(depth)]
    for _ in range(width):
        for row in range(depth):
            if i < ncols:
                col = df.columns[i]
                list_[row].extend(['<b>{}</b>'.format(col), df[col].count(), df.dtypes[i]])
            i += 1

    print('{} entries, {} columns'.format(len(df), ncols))
    pp_listOflist(list_)


def pp_counts(series, rows=1, caption=None):
    if caption: pp_bold(caption)
    list_ = [(k, '{:.4f}'.format(v)) for k, v in series.to_dict().items()] 
    dict_ = OrderedDict(sorted(list_, key=lambda x: x[0]))
    pp_dict(dict_, rows)


def pp_progress(s):
    sys.stdout.write('\r{}'.format(s))
    sys.stdout.flush()


Implement simple web page cache.

In [26]:
import os
import pickle
import requests

_cache = None
_cache_dir = "./cache"
_cache_index = os.path.join(_cache_dir, 'index.pkl')
_cache_counter = 0

def cache_init():
    global _cache
    if _cache == None:
        if os.path.exists(_cache_index):
            with open(_cache_index, 'rb') as fd:
                _cache = pickle.load(fd)
        else:
            _cache = {}
    return _cache

def cache_get(key):
    return cache_init().get(key, '')

def cache_add(key, value):
    global _cache_counter
    cache = cache_init()
    cache[key] = value
    _cache_counter += 1
    if _cache_counter % 100 == 0:
        cache_commit()

def cache_commit():
    if not _cache == None:
        with open(_cache_index, 'wb') as fd:
            pickle.dump(_cache, fd)

In [437]:
def get_page(url, useCache=True):
    """Get a web page."""

    # Check if we have this page
    
    filename = cache_get(url)
    if useCache and filename and os.path.exists(filename):
        with open(filename, 'rb') as fd:
            return fd.read()

    # Otherwise, download the page ...
    
    r = requests.get(url, timeout=10)
    r.raise_for_status()
    
    # ... and cache it

    global _cache_dir
    if not os.path.isdir(_cache_dir):
        os.mkdir(_cache_dir)
        
    if not filename:
        filename = os.path.join(_cache_dir, uuid.uuid4().hex + '.html')

    with open(filename, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=4096):
            fd.write(chunk)

    cache_add(url, filename)
    
    return r.text

In [422]:
import datetime
import re
import time
from bs4 import BeautifulSoup
from geojson import Point

### Web Scraping

Connect to the MongoDB database.

In [439]:
import pymongo
client = pymongo.MongoClient('ec2-34-198-246-43.compute-1.amazonaws.com', 27017)
db = client.renthop2
collection = db.listings
pp_bold('{} listings'.format(collection.count()))

Scrape the pages and add new listings.

In [440]:
def extract_details(listing):
    html_doc = get_page(listing['url'])
    soup = BeautifulSoup(html_doc, 'lxml')
    body = soup.find(id='listing-details-body-premap')
    if not body:
        print('warning: body not found: {}'.format(listing['url']), file=sys.stderr)
        return

    for div in body.find_all('div', recursive=False):
        desc = div.find('div', id='description')
        if desc:
            listing['description'] = desc.find_next_sibling().text.strip()
            continue

        features_div = div.find('div', text=re.compile('Features & Amenities'))
        if features_div:
            features = []
            for div in features_div.find_next_sibling().find_all('div'):
                table = div.find('table')
                if table:
                    feature = table.tr.find_all('td')[1].text.strip()
                    if 'Featured' != feature:
                        features.append(feature)
            listing['features'] = '\n'.join(features)
            continue

        for table in div.find_all('table'):
            for tr in table.find_all('tr'):
                for td in tr.find_all('td'):
                    if 'Studio' in td.text:
                        listing['bedrooms'] = 0.0
                    elif 'Bed' in td.text:
                        listing['bedrooms'] = float(td.text.strip().split()[0])
                    elif 'Bath' in td.text:
                        listing['bathrooms'] = float(td.text.strip().split()[0])


In [443]:
URL_RENTHOP = 'https://www.renthop.com/search/nyc?min_price=0&max_price=50000&sort=hopscore&page={}'

# Identify old listings

seen = set([r['listing_id'] for r in collection.find({}, {'_id':0, 'listing_id':1})])

# Scrape new listings

for pageno in range(925, 4000):
    # Download page

    url = URL_RENTHOP.format(pageno)
    html_doc = get_page(url, useCache=False)
    soup = BeautifulSoup(html_doc, 'lxml')
    pp_progress('Scraping page {} '.format(url))

    # Verify page number
    
    page_box = soup.find('input', id='page_input_box')
    if page_box['value'] != str(pageno):
        print('error: incorrect page_input_box value: {}', page_box['value'], file=sys.stderr)
        break

    # Find listings

    listings_divs = soup.find_all('div', class_='search-listing')
    if not listings_divs:
        print('error: no listings on page {}'.format(pageno), file=sys.stderr)
        break
        
    # Extract listings
    
    bulk = collection.initialize_ordered_bulk_op()
    for div in listings_divs:
        listing_id = int(div['listing_id'])
        if listing_id in seen:
            continue
            
        listing = { 
            'created' : unicode(datetime.datetime.utcnow()).split('.')[0],
        }
        listing['listing_id'] = listing_id
        listing['latitude'] = float(div['latitude'])
        listing['longitude'] = float(div['longitude'])
        listing['loc'] = Point((listing['longitude'], listing['latitude']))
        
        title = div.find('a', class_='listing-title-link')
        listing['url'] = title['href']
        listing['title'] = title.string
        
        price = div.find(id='listing-{}-price'.format(listing_id))
        listing['price'] = int(re.sub('[^0-9]', '', price.string))
        
        # Extract listing details
        
        pp_progress('Scraping page {}: {}'.format(pageno, listing['url']))
        extract_details(listing)
        if 'bedrooms' in listing: # ignore shares
            bulk.insert(listing)
            seen.add(listing_id)
#            time.sleep(0.25)

    # Save listings

    try:
        bulk.execute()
        cache_commit()
    except pymongo.errors.InvalidOperation as e:
        if str(e) == 'No operations to execute':
            pass
        
print('\nDone!')

Scraping page 1383: https://www.renthop.com/listings/east-16th-street/6p/8661183

Scraping page 1432: https://www.renthop.com/listings/classon/15l/8740631

Scraping page 1433: https://www.renthop.com/listings/west-65th-street/4a/8348997

Scraping page 1516: https://www.renthop.com/listings/dahill-road/5t/8503853

Scraping page 1556: https://www.renthop.com/listings/31st-street/3b/8695181

...

Scraping page 2443: https://www.renthop.com/listings/e-54-street/4a/8705808

Scraping page 2443: https://www.renthop.com/listings/maiden-lane/506/8570902

Scraping page 2463: https://www.renthop.com/listings/220-w-107th/4d/8721591

Scraping page https://www.renthop.com/search/nyc?min_price=0&max_price=50000&sort=hopscore&page=3174 
Done!


error: incorrect page_input_box value: {} 3173


In [434]:
import pandas as pd
import numpy as np
columns = ['listing_id', 'bedrooms', 'bathrooms', 'features', 'price']
df = pd.DataFrame(list(collection.find({}, columns)), columns=columns)
print(len(df))

217


In [444]:
len(seen)

95126

In [435]:
df.sort_values(by='listing_id')

,listing_id,bedrooms,bathrooms,features,price
71,4017636,2.0,1.0,,2500
76,6361252,1.0,1.0,Cats Allowed\nDogs Allowed\nElevator\nFitness ...,3650
50,6391866,0.0,1.0,Private Outdoor Space\nElevator\nLaundry In Bu...,2295
180,6469427,1.0,1.0,No Fee\nElevator\nLaundry In Building\nDishwas...,3600
63,6499842,0.0,1.0,Reduced Fee,1650
64,6676453,1.0,1.0,Elevator\nFireplace\nDishwasher\nHardwood Floors,2825
67,6908537,4.0,2.0,Laundry In Unit\nElevator\nOWN THE FLOOR\nGOUR...,6900
5,7003330,0.0,1.0,Reduced Fee\nCats Allowed\nDogs Allowed,1655
57,7087544,0.0,1.0,Cats Allowed\nDogs Allowed\nLaundry In Buildin...,1750
153,7116578,3.0,1.0,No Fee\nParking Space\nCats Allowed\nDogs Allo...,3600


In [414]:
collection.drop()